In [ ]:
import cv2
!nvidia-smi


In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)  # CUDA version that PyTorch was built with
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())


In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
# Git clone method (for development)

# %cd {HOME}
# !git clone github.com/ultralytics/ultralytics
# %cd {HOME}/ultralytics
# !pip install -e .

# from IPython import display
# display.clear_output()

# import ultralytics
# ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
from torch.serialization import add_safe_globals
from ultralytics.nn.tasks import DetectionModel

add_safe_globals([DetectionModel])


## CLI Basics

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

## Inference with Pre-trained COCO Model

###  CLI

`yolo mode=predict` runs YOLOv8 inference on a variety of sources, downloading models automatically from the latest YOLOv8 release, and saving results to `runs/predict`.

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

In [ ]:
%cd {HOME}
Image(filename='runs/detect/predict/dog.jpeg', height=600)

###  Python SDK



In [ ]:
model = YOLO(f'{HOME}/yolov8n.pt')
results = model.predict(source='https://media.roboflow.com/notebooks/examples/dog.jpeg', conf=0.25)

In [ ]:
results[0].boxes.xyxy

In [ ]:
results[0].boxes.conf

In [ ]:
results[0].boxes.cls

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets
!pip install --upgrade urllib3 requests_toolbelt


!pip install roboflow


from roboflow import Roboflow
rf = Roboflow(api_key="6ens9GO4bDTQdLm9V25F")
project = rf.workspace("eshan-shukla-alkkq").project("crop-disease-qhu91")
version = project.version(1)
dataset = version.download("yolov8")
                
                
                

In [ ]:
#!pip install roboflow

#from roboflow import Roboflow
#rf = Roboflow(api_key="6ens9GO4bDTQdLm9V25F")
#project = rf.workspace("eshan-shukla-alkkq").project("basketball-2bsnt")
#version = project.version(1)
#dataset = version.download("yolov4pytorch")
                
!pip install roboflow


from roboflow import Roboflow
rf = Roboflow(api_key="6ens9GO4bDTQdLm9V25F")
project = rf.workspace("eshan-shukla-alkkq").project("crop-disease-qhu91")
version = project.version(2)
dataset = version.download("yolov8")

## Custom Training

In [ ]:
%cd {HOME}

#!yolo task=detect mode=train model=yolov8s-seg.pt data={dataset.location}/data.yaml epochs=20 imgsz=640 batch=4 workers=2 plots=True
#!yolo task=detect mode=train model=yolov8n.pt data={dataset.location}/data.yaml epochs=25 imgsz=640 batch=4 workers=2 plots=True
!yolo task=detect mode=train model=yolov8s.pt data="/home/theperson/YOLO/Projects/Basketball/Basketball hoop images.v1i.yolov8/data.yaml" epochs=15 imgsz=640 batch=4 workers=2 plots=True

#!yolo task=detect mode=train model=yolov5n-cls.pt data={dataset.location}/data.yaml epochs=25 imgsz=640 batch=4 workers=2 plots=True


In [ ]:
#!ls {HOME}/runs/detect/train9/
!ls /home/theperson/YOLO/Train/runs/segment/train6/
#/home/theperson/YOLO/Train/runs/segment/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train6/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train6/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train6/val_batch0_pred.jpg', width=600)

## Validate Custom Model

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train9/weights/best.pt data={dataset.location}/data.yaml

## Inference with Custom Model

In [ ]:
%cd {HOME}
#!yolo task=detect mode=predict model={HOME}/runs/detect/train8/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True
#!yolo task=detect mode=predict model={HOME}/runs/detect/train9/weights/best.pt conf=0.25 source=/home/theperson/YOLO/Projects/Basketball/3602205297-preview.mp4 save=True
#!yolo task=detect mode=predict model="/home/theperson/Vajra/MyProjects/Basketball_ Tracking/AI-Basketball-Shot-Detection-Tracker-master/ball_hoop/v8/best (1).pt" conf=0.25 source=/home/theperson/YOLO/Projects/Basketball/3602205297-preview.mp4 save=True

#live cam
!yolo task=detect mode=predict model={HOME}/runs/detect/train8/weights/best.pt conf=0.25 source=0 save=True save_txt=False save_conf=False save_crop=False

#!yolo task=detect mode=predict model={HOME}/runs/detect/train8/weights/best.pt conf=0.25 source=0 save=True show=True
#cv2.imshow('YOLO Detection', frame)



In [ ]:
import cv2
from ultralytics import YOLO

model=YOLO("/home/theperson/YOLO/Train/runs/segment/train6/weights/best.pt")

cap=cv2.VideoCapture(0) 

while True:
    ret, frame=cap.read()
    if not ret:
        break
    results=model.predict(frame) 

    frame=results.plot()  

    cv2.imshow('YOLOv8 Detection', frame)

    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

cap.release()
#cv2.destroyAllWindows()


**NOTE:** Let's take a look at few results.

In [ ]:
import glob
from IPython.display import Image, display

# Define the base path where the folders are located
base_path = '/home/theperson/YOLO/Train/runs/segment'

# List all directories that start with 'predict' in the base path
subfolders = [os.path.join(base_path, d) for d in os.listdir(base_path)
              if os.path.isdir(os.path.join(base_path, d)) and d.startswith('predict')]

# Find the latest folder by modification time
latest_folder = max(subfolders, key=os.path.getmtime)

image_paths = glob.glob(f'{latest_folder}/*.jpg')[:3]

# Display each image
for image_path in image_paths:
    display(Image(filename=image_path, width=600))
    print("\n")

In [ ]:
version = project.version(2)
version.deploy("yolov8-type", "/home/theperson/YOLO/runs/segment/train3")